In [186]:
import pandas as pd
import numpy as np
import pytz
from datetime import datetime

In [475]:
fb = pd.read_csv('/Users/fnokeke/Desktop/TP__fb_stats.csv')
fb.shape

(16686, 16)

### Unique participants

In [476]:
len(fb.worker_id.unique())

146

### Remove developer stats

In [477]:
fb = fb.loc[fb['worker_id'] != 'Fabian']
fb = fb.loc[fb['worker_id'] != 'chief scientist']
len(fb.worker_id.unique())

144

### Extract only date

In [478]:
fb.index = pd.to_datetime(fb['created_at'], utc=True)
fb.index = fb.index.tz_localize('UTC').tz_convert('Asia/Calcutta')
fb['only_date'] = fb.index.date
fb['only_date'] = fb.only_date.apply(lambda x: x.strftime('%Y-%m-%d'))
fb = fb.drop('created_at', 1)
fb = fb.reset_index()
fb.head(3)

,created_at,id,worker_id,total_seconds,total_opens,time_spent,time_open,ringer_mode,current_experiment_group,current_fb_max_mins,current_fb_max_opens,current_treatment_start,current_followup_start,current_logging_stop,daily_reset_hour,screen_logs,only_date
0,2017-04-07 15:12:25.032209+05:30,39,A2KOXR5OXIFUIW,300,0,30,3,normal,1,10,2,2017-04-16 00:00:00,2017-04-24 00:00:00,2017-05-02 00:00:00,0,NaN,2017-04-07
1,2017-04-07 15:12:25.414346+05:30,40,A2KOXR5OXIFUIW,300,0,30,3,normal,1,10,2,2017-04-16 00:00:00,2017-04-24 00:00:00,2017-05-02 00:00:00,0,NaN,2017-04-07
2,2017-04-07 15:12:26.249622+05:30,41,A2KOXR5OXIFUIW,300,0,30,3,normal,1,10,2,2017-04-16 00:00:00,2017-04-24 00:00:00,2017-05-02 00:00:00,0,NaN,2017-04-07


### Specific worker ids to use

In [479]:
wids_to_use = """
A23RGS48XSHCT5
A1BP3A40FCVRRU
A3QWTKQGX8C9YS
ABPWNQ6VWH8OI
A1M6A01CROTF3K
A3LX64F2JXHTXJ
A11W99OO65FV4K
A3URYXX9F43WQ0
A252JVFQ7DCO40
A2SSYBUT9ZM0G9
A2L9YY0SUCO113
A3DS9DP2JE8I4Z
A2U7SC4R0K7K3M
A2958TQAW43MP6
AURF17VKAGMY5
AS5SGELZJDP48
A2FB6X54WMYEKQ
A2F3DLC1E57DOE
AYMJAMX7Z7YHG
ANPZ22WUNLOX1
A18EWG6CNKA02J
A1CQK70BFYLIG5
A12YW4OVKB6EBR
A3FRMGQHG2IXSX
A4A2TC8YA24E6
"""

wids_to_use = [x for x in wids_to_use.split("\n") if x != '']
len(wids_to_use)

25

### Group stats

In [482]:
 cols = [
 'worker_id',
 'time_spent',
 'time_open',
 'only_date'
]
df = fb[cols]
df.head(2)

,worker_id,time_spent,time_open,only_date
0,A2KOXR5OXIFUIW,30,3,2017-04-07
1,A2KOXR5OXIFUIW,30,3,2017-04-07


In [483]:
wid = 'A1BP3A40FCVRRU'

In [484]:
wfilter = df[df['worker_id'] == wid]
rdf = wfilter.sort_index().groupby(['worker_id', 'only_date']).apply(max)
rdf

worker_id  time_spent  time_open   only_date
worker_id      only_date                                                    
A1BP3A40FCVRRU 2017-04-09  A1BP3A40FCVRRU        2105         14  2017-04-09
               2017-04-10  A1BP3A40FCVRRU         345          2  2017-04-10
               2017-04-11  A1BP3A40FCVRRU         480          4  2017-04-11
               2017-04-12  A1BP3A40FCVRRU         460          2  2017-04-12
               2017-04-13  A1BP3A40FCVRRU         125          2  2017-04-13
               2017-04-14  A1BP3A40FCVRRU         995          3  2017-04-14
               2017-04-15  A1BP3A40FCVRRU        1395          2  2017-04-15
               2017-04-16  A1BP3A40FCVRRU        1395          2  2017-04-16
               2017-04-17  A1BP3A40FCVRRU         625          2  2017-04-17
               2017-04-18  A1BP3A40FCVRRU         625          3  2017-04-18

In [465]:
rdf = rdf.drop('only_date', 1)
rdf = rdf.drop('worker_id', 1)
rdf = rdf.reset_index()
rdf

,worker_id,only_date,time_spent,time_open
0,A1BP3A40FCVRRU,2017-04-10,345,2
1,A1BP3A40FCVRRU,2017-04-11,480,4
2,A1BP3A40FCVRRU,2017-04-12,460,2
3,A1BP3A40FCVRRU,2017-04-13,125,2
4,A1BP3A40FCVRRU,2017-04-14,995,3
5,A1BP3A40FCVRRU,2017-04-15,1395,2
6,A1BP3A40FCVRRU,2017-04-16,1395,2
7,A1BP3A40FCVRRU,2017-04-17,625,2
8,A1BP3A40FCVRRU,2017-04-18,625,3


### Drop specific columns

In [414]:
c1 = df.only_date == '2017-04-09'
c2 = df.worker_id == wid
mask = ~(c1 & c2)

In [415]:
print 'Before drop: {}'.format(df.shape)
df = df.loc[mask]
print 'After drop: {}'.format(df.shape)

Before drop: (16559, 4)
After drop: (16548, 4)


In [466]:
stats = rdf.describe()
dd = stats.to_dict()

In [467]:
stats

,time_spent,time_open
count,9.000000,9.000000
mean,716.111111,2.444444
std,450.643830,0.726483
min,125.000000,2.000000
25%,460.000000,2.000000
50%,625.000000,2.000000
75%,995.000000,3.000000
max,1395.000000,4.000000


### Combine personal average across all groups

In [485]:
time_spent_list = []
time_open_list = []

time_spent_tuples = []
time_open_tuples = []


for worker_id in df.worker_id.unique():
    if worker_id not in wids_to_use:
        continue
    
    wfilter = df[df['worker_id'] == worker_id]
    rdf = wfilter.sort_index().groupby(['worker_id', 'only_date']).apply(max)
    stats = rdf.describe()
    
    dd = stats.to_dict()
    avg_time_spent = dd['time_spent']['mean']
    avg_time_open = dd['time_open']['mean']
        
    time_spent_list.append(avg_time_spent)
    time_open_list.append(avg_time_open)

    count_time_spent = dd['time_spent']['count']
    count_time_open = dd['time_open']['count']
    
    time_spent_tuples.append((avg_time_spent, count_time_spent))
    time_open_tuples.append((avg_time_open, count_time_open))


In [486]:
print "len(time_spent_list): {}".format(len(time_spent_list))
print "len(time_open_list): {}".format(len(time_open_list))

len(time_spent_list): 25
len(time_open_list): 25


In [487]:
sum(time_spent_list)*1.0 / len(time_open_list)

2836.1941269841273

In [488]:
sum(time_open_list)*1.0 / len(time_open_list)

12.058190476190475

In [489]:
time_open_list

[11.0,
 3.6000000000000001,
 21.800000000000001,
 5.5,
 27.800000000000001,
 7.4000000000000004,
 37.600000000000001,
 5.2000000000000002,
 18.0,
 7.2000000000000002,
 1.7,
 12.555555555555555,
 15.555555555555555,
 41.888888888888886,
 16.111111111111111,
 10.666666666666666,
 1.3333333333333333,
 15.222222222222221,
 3.0,
 7.0,
 3.5714285714285716,
 17.666666666666668,
 2.8333333333333335,
 3.25,
 4.0]

In [490]:
time_open_tuples

[(11.0, 10.0),
 (3.6000000000000001, 10.0),
 (21.800000000000001, 10.0),
 (5.5, 10.0),
 (27.800000000000001, 10.0),
 (7.4000000000000004, 10.0),
 (37.600000000000001, 10.0),
 (5.2000000000000002, 10.0),
 (18.0, 10.0),
 (7.2000000000000002, 10.0),
 (1.7, 10.0),
 (12.555555555555555, 9.0),
 (15.555555555555555, 9.0),
 (41.888888888888886, 9.0),
 (16.111111111111111, 9.0),
 (10.666666666666666, 9.0),
 (1.3333333333333333, 9.0),
 (15.222222222222221, 9.0),
 (3.0, 4.0),
 (7.0, 5.0),
 (3.5714285714285716, 7.0),
 (17.666666666666668, 6.0),
 (2.8333333333333335, 6.0),
 (3.25, 4.0),
 (4.0, 5.0)]

In [491]:
time_spent_tuples

[(1854.0, 10.0),
 (855.0, 10.0),
 (2695.0, 10.0),
 (909.5, 10.0),
 (2636.0, 10.0),
 (1011.5, 10.0),
 (4602.0, 10.0),
 (3203.5, 10.0),
 (4348.5, 10.0),
 (733.5, 10.0),
 (240.5, 10.0),
 (4148.8888888888887, 9.0),
 (6996.666666666667, 9.0),
 (5198.333333333333, 9.0),
 (9153.8888888888887, 9.0),
 (5015.0, 9.0),
 (97.777777777777771, 9.0),
 (5455.0, 9.0),
 (137.5, 4.0),
 (1322.0, 5.0),
 (1215.7142857142858, 7.0),
 (7775.833333333333, 6.0),
 (405.0, 6.0),
 (621.25, 4.0),
 (273.0, 5.0)]